In [1]:
from alternating_least_square import AlternatingLeastSquare

In [2]:

data_dir = '../data/ml-25m/ratings.csv'
als_25m_data = AlternatingLeastSquare(data_dir, embedding_dim=2)
als_25m_data.data_indexing()

In [3]:

als_25m_data.train_test_split()

In [4]:
user_biases, item_biases ,losses, rmse, losses_test, rmses_test= als_25m_data.alternating_least_square(als_25m_data.data_by_user_train, als_25m_data.data_by_movie_train, lambd=0.1, gamma=0.001, epochs = 2)

Training: 100%|██████████| 2/2 [05:01<00:00, 150.90s/epoch, test_rmse=0.882]


In [6]:
movies_dir = '../data/ml-25m/movies.csv'
als_25m_data.recommendation_for_new_user(movies_dir, lambd=0.1, gamma=0.001)

MemoryError: Unable to allocate 71.5 GiB for an array with shape (162541, 59047) and data type float64